In [1]:
%load_ext autoreload
%autoreload 1

In [17]:
import os
import pickle
from metric import compute_metric


def check_one_set(data_dir, pkl_path, thres):
    results = sorted(os.listdir(pkl_path))
    num_split = len(results)
    dataset = pd.read_csv(data_dir, sep='\t')
    dataset['seqlen'] = dataset['text'].apply(lambda x: len(x))
    for i in range(num_split):
        pkl_dir = f'{pkl_path}/{results[i]}'
        if not os.path.exists(pkl_dir):
            print('Not existing', pkl_dir)
            continue
        with open(pkl_dir, 'rb') as fr:
            pkl = pickle.load(fr)
            outputs, labels = pkl
#            auroc_val, precision, recall = compute_metric(labels[:, 1], outputs[:, 1], thres=thres)
        dataset[i] = outputs[:, 1]

    column_names = range(num_split)
    averages = dataset[column_names].mean(axis=1)
#    print('Mean')
    auroc_val, precision, recall = compute_metric(labels[:, 1], averages, thres=thres)
    
    d_list = [dataset[(i*20 < dataset['seqlen']) & (dataset['seqlen'] <= (i + 1) * 20)] for i in range(3)]

    for idx, d in enumerate(d_list):
        print(f'Sequence Length from {idx * 20} to {(idx + 1) * 20}')
        if (len(d) == 0):
            continue
        for i in range(num_split):
            labels, outputs = d['label'].tolist(), d[i].tolist()
            auroc_val, precision, recall = compute_metric(labels, outputs)
        averages = d[column_names].mean(axis=1)
#        print('Mean')
        auroc_val, precision, recall = compute_metric(labels, averages)


In [20]:
#查看单组结果
runtime_list = sorted(os.listdir('result'))[-1:]
for runtime in runtime_list[::-1]:
    print('\n' + runtime)
    for idx in range(1,5):
        data_dir = f'../ACP_datasets/ACP-Mixed-80-test.tsv'
        name = f'model{idx}'
        print(name)
        pkl_path = f'result/{runtime}/{name}'
        if os.path.exists(pkl_path):
            check_one_set(data_dir, pkl_path, thres=0.5)
        else:
            print(f"{pkl_path} not exists")


2023-09-24_21-17-06
model1
0.7951	0.7571	0.7213	0.5967	0.8619
Sequence Length from 0 to 20
0.8305	0.8788	0.8333	0.6549	0.8786
0.8305	0.8571	0.7917	0.6488	0.8631
0.7966	0.8286	0.75	0.5786	0.856
0.8305	0.8571	0.7917	0.6488	0.8702
Sequence Length from 20 to 40
0.8367	0.7419	0.6923	0.7167	0.8846
0.8571	0.7667	0.7308	0.7485	0.9013
0.8367	0.7419	0.6923	0.7167	0.888
0.8571	0.7667	0.7308	0.7485	0.8896
Sequence Length from 40 to 60
0.5	0.0	0.5455	-0.2548	0.4545
0.5833	0.0	0.6364	-0.2132	0.3636
0.4167	0.0	0.4545	-0.3015	0.2727
0.5	0.0	0.5455	-0.2548	0.3636
model2
0.7951	0.7812	0.7705	0.5909	0.8788
Sequence Length from 0 to 20
0.7627	0.8182	0.75	0.5159	0.8643
0.7797	0.775	0.625	0.5369	0.8976
0.7797	0.7895	0.6667	0.5374	0.8476
0.7797	0.8056	0.7083	0.5408	0.8762
Sequence Length from 20 to 40
0.7959	0.8095	0.8462	0.5902	0.8763
0.8163	0.7917	0.8077	0.6327	0.898
0.8367	0.8	0.8077	0.6761	0.9047
0.8163	0.7917	0.8077	0.6327	0.9064
Sequence Length from 40 to 60
0.75	0.0	0.8182	-0.1348	0.1818
0.75	0.0	0.8

In [9]:
from metric import compute_metric_labelwise, compute_metric_score
import numpy as np
import pickle
folder = 'result/2023-09-25_10-34-34'
labels, outputs = [], []
for k in range(1, 3):
    result_file = f'{folder}/model3/{k}.pkl'
    res = pickle.load(open(result_file, 'rb'))
    output, label = res
    labels.append(label)
    outputs.append(output)
labels = np.vstack(labels)
outputs = np.vstack(outputs)
print(labels.shape)
#outputs
compute_metric_labelwise(labels, outputs)
compute_metric_score(labels, outputs)

(43, 7)
0.822	0.744	0.7	0.55	0.792	0.503
0.963	0.884	0.84	0.81	0.894	0.774
0.918	0.814	0.81	0.818	0.81	0.628
0.857	0.814	0.739	0.75	0.81	0.642
0.885	0.86	0.789	0.846	0.833	0.717
0.921	0.837	0.765	0.852	0.788	0.657
------Macro------
Macro accuracy 0.8255813953488372
Macro precision 0.7291960389154781
Macro recall 0.8183201410214755
Macro f1-score 0.7696954361804966
------Micro------
Micro accuracy 0.8039867109634552
Micro precision 0.7432432432432432
Micro recall 0.8396946564885496
Micro f1-score 0.7885304659498208
------Hamming loss------
Hamming loss 0.19601328903654486


0.8255813953488372

In [5]:
labels.shape, outputs.shape

((759, 7), (759, 7))